In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import math
import unicodedata
import re
import torch.nn as nn
import torch.nn.functional as F
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
import CustomBert
from WordPieceTokenizer import WordPieceTokenizer as Tokenizer
import os
from datasets import load_dataset
from PretrainDataset import TokenizedDataset
from PretrainDataset import CustomDataCollatorForMLM

In [2]:
def group_texts(examples, tokenizer, MAX_SEQUENCE_LENGTH):
    print(f"--- group_texts 함수 호출됨 ---")
    print(f"examples['text'] 타입: {type(examples['text'])}")
    print(f"examples['text'] 길이: {len(examples['text'])}")
    if examples['text']:
        print(f"examples['text'] 첫 번째 요소 타입: {type(examples['text'][0])}")
        print(f"examples['text'] 첫 번째 요소 내용 (앞 50자): '{examples['text'][0][:50]}'")
    else:
        print(f"examples['text']가 비어있습니다.")

    concatenated_text = " ".join(examples["text"])
    
    if not concatenated_text.strip():
        print(f"group_texts 경고: 연결된 텍스트가 비어있거나 공백만 있습니다. 텍스트: '{concatenated_text}'")

    encoded_output = tokenizer.encode(
        concatenated_text,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        padding=False,
        add_special_tokens=True
    )
    
    if len(encoded_output['input_ids']) <= 2:
        print(f"--- group_texts 짧은 시퀀스 디버그 ---")
        print(f"경고: Encoded input_ids 길이가 예상보다 매우 짧습니다: {len(encoded_output['input_ids'])}")
        print(f"연결된 텍스트 길이: {len(concatenated_text)}")
        print(f"연결된 텍스트 예시 (앞 100자): '{concatenated_text[:100]}'")
        print(f"인코딩된 input_ids 예시: {encoded_output['input_ids']}")
        print(f"인코딩된 input_ids를 다시 디코드: '{tokenizer.decode(encoded_output['input_ids'])}'")
        print(f"--- 디버그 종료 ---\n")

    return encoded_output

In [3]:
tokenizer = Tokenizer(vocab_file_path="Pretrained/vocab.txt",do_lower_case=False,strip_accents=False,clean_text=True)
VOCAB_SIZE = tokenizer.get_vocab_size()
MAX_SEQUENCE_LENGTH = 128
BATCH_SIZE = 16

datasetsPath = 'datasets/'
PREPROCESSED_TEXT_DIR = f'{datasetsPath}preprocess_wiki_text'

text_files = [os.path.join(PREPROCESSED_TEXT_DIR, f) for f in os.listdir(PREPROCESSED_TEXT_DIR) if f.endswith('.txt')]

if not text_files:
    exit()
print(f'총 {len(text_files)}개의 텍스트 파일 로드 시작...')
raw_dataset = load_dataset("text", data_files={"train": text_files}, split="train")
print(f"원시 데이터셋 로드 완료. 총 {len(raw_dataset)}개의 샘플.")
    
print('데이터셋 토큰화 및 청킹 시작...')
tokenized_dataset = raw_dataset.map(
    group_texts,
    batched=True,
    num_proc=1,
    remove_columns=["text"],
    fn_kwargs={"tokenizer":tokenizer,"MAX_SEQUENCE_LENGTH":MAX_SEQUENCE_LENGTH},
    desc=f"맵핑 데이터셋 (토큰화 및 청킹, 최대 길이 {MAX_SEQUENCE_LENGTH})"
)
print(len(tokenized_dataset))

총 8개의 텍스트 파일 로드 시작...
원시 데이터셋 로드 완료. 총 702964개의 샘플.
데이터셋 토큰화 및 청킹 시작...


맵핑 데이터셋 (토큰화 및 청킹, 최대 길이 128):   0%|          | 0/702964 [00:00<?, ? examples/s]

--- group_texts 함수 호출됨 ---
examples['text'] 타입: <class 'list'>
examples['text'] 길이: 1000
examples['text'] 첫 번째 요소 타입: <class 'str'>
examples['text'] 첫 번째 요소 내용 (앞 50자): '제임스 지미 카터 주니어 미국의 대통령 지낸 미국의 정치인이다 민주당 소속으로 년부터 년까'
--- group_texts 함수 호출됨 ---
examples['text'] 타입: <class 'list'>
examples['text'] 길이: 1000
examples['text'] 첫 번째 요소 타입: <class 'str'>
examples['text'] 첫 번째 요소 내용 (앞 50자): '트롬쇠 troms 노르웨이 북부 트롬스주 troms 위치한 도시이다 현재 명의 주민이 있다'
--- group_texts 함수 호출됨 ---
examples['text'] 타입: <class 'list'>
examples['text'] 길이: 1000
examples['text'] 첫 번째 요소 타입: <class 'str'>
examples['text'] 첫 번째 요소 내용 (앞 50자): '연호 남제 건무 북위 태화 기년 남제 명제 북위 효문제 신라 소지 마립간 고구려 문자명왕 '
--- group_texts 함수 호출됨 ---
examples['text'] 타입: <class 'list'>
examples['text'] 길이: 1000
examples['text'] 첫 번째 요소 타입: <class 'str'>
examples['text'] 첫 번째 요소 내용 (앞 50자): '북해와 발트해를 둘러싸고 있는 섬과 대륙 지역을 촬영한 합성 위성사진 북유럽 유럽의 북부 '
--- group_texts 함수 호출됨 ---
examples['text'] 타입: <class 'list'>
examples['text'] 길이: 1000
examples['text'] 첫 번째 요

In [4]:
train_dataset = TokenizedDataset(tokenized_dataset)

data_collator = CustomDataCollatorForMLM(tokenizer=tokenizer, mlm_probability=0.15)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=data_collator,
    num_workers=os.cpu_count()
)

print(f"PyTorch DataLoader 생성 완료. 배치 크기: {BATCH_SIZE}")
print(f"총 훈련 배치 수: {len(train_dataloader)}")

PyTorch DataLoader 생성 완료. 배치 크기: 16
총 훈련 배치 수: 5624


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

HIDDEN_SIZE = 768
NUM_HIDDEN_LAYERS = 12
NUM_ATTENTION_HEADS = 12
INTERMEDIATE_SIZE = 3072
TYPE_VOCAB_SIZE = 2
DROPOUT_PROB = 0.1

model = CustomBert.CustomBertForMaskedLM(
    vocab_size=VOCAB_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_hidden_layers=NUM_HIDDEN_LAYERS,
    num_attention_heads=NUM_ATTENTION_HEADS,
    intermediate_size=INTERMEDIATE_SIZE,
    max_position_embeddings=MAX_SEQUENCE_LENGTH,
    type_vocab_size=TYPE_VOCAB_SIZE,
    dropout_prob=DROPOUT_PROB
)
model.to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Custom Bert 모델 초기화 완료. 총 학습 가능 파라미터 수 : {num_params}')

Custom Bert 모델 초기화 완료. 총 학습 가능 파라미터 수 : 110946560


In [7]:
EPOCHS = 1
LEARNING_RATE = 2e-6
WEIGHT_DECAY = 0.01
WARMUP_STEPS = 5000
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
total_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS,num_training_steps=total_steps)

In [8]:
print(f"\n<--- 학습 시작 ---> ({EPOCHS} 에폭)")
model.train()

CHECKPOINT_DIR = "pretrain_checkpoints"
os.makedirs(CHECKPOINT_DIR,exist_ok=True)

for e in range(EPOCHS):
    loss_sum = 0
    progress_bar = tqdm(train_dataloader, desc=f"Pre-train Epoch {e+1}")

    for step, batch in enumerate(progress_bar):
        batch = {k: v.to(device) for k,v in batch.items()}

        for k, v in batch.items():
            if torch.isnan(v).any():
                print(f"Warning: NaN found in batch[{k}] at step {step}")
            if torch.isinf(v).any():
                print(f"Warning: Inf found in batch[{k}] at step {step}")
        
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            token_type_ids=batch["token_type_ids"],
            labels=batch["labels"]
        )
        loss = outputs["loss"]

        # if loss is not None and torch.isnan(loss):
        #     print(f"NaN loss detected at step {step}. Inspecting prediction_scores and labels.")
        #     print(f"prediction_scores shape: {outputs['logits'].shape}")
        #     print(f"prediction_scores min: {outputs['logits'].min().item()}")
        #     print(f"prediction_scores max: {outputs['logits'].max().item()}")
        #     print(f"prediction_scores mean: {outputs['logits'].mean().item()}")
        #     print(f"prediction_scores std: {outputs['logits'].std().item()}")
        #     print(f"Labels shape: {batch['labels'].shape}")
        #     print(f"Labels unique values: {torch.unique(batch['labels'])}")
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        loss_sum += loss.item()
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_train_loss = loss_sum / len(train_dataloader)
    print(f"Pre-train Epoch {e+1} 완료. 평균 학습 손실: {avg_train_loss:.4f}")

    MODEL_SAVE_PATH = os.path.join(CHECKPOINT_DIR,f"epoch_{e+1}_pytorch_model.pt")
    torch.save(model.state_dict(),MODEL_SAVE_PATH)
    print(f"모델 가중치 '{MODEL_SAVE_PATH}' 저장 완료.")
print("\n<--- 학습 완료 --->")


<--- 학습 시작 ---> (1 에폭)


Pre-train Epoch 1:   0%|          | 0/5624 [00:37<?, ?it/s]

KeyboardInterrupt: 